In [56]:
# get data from arxiv api 
# https://gist.github.com/jozefg/c2542f51a0b9b3f6efe528fcec90e334

import requests
url = 'http://export.arxiv.org/api/query?search_query=cat:cs.SD&sortBy=submittedDate&sortOrder=descending&max_results=50'
response = requests.get(url)
xml_data = response.text  # data.read().decode('utf-8')
print(xml_data)

<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dcat%3Acs.SD%26id_list%3D%26start%3D0%26max_results%3D50" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=cat:cs.SD&amp;id_list=&amp;start=0&amp;max_results=50</title>
  <id>http://arxiv.org/api/Uk4hOarnIHkcx3INbQB+qNBCFEo</id>
  <updated>2024-02-21T00:00:00-05:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">13172</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">50</opensearch:itemsPerPage>
  <entry>
    <id>http://arxiv.org/abs/2402.13236v1</id>
    <updated>2024-02-20T18:50:25Z</updated>
    <published>2024-02-20T18:50:25Z</published>
    <title>Towards audio language modeling - an overview</title>


In [57]:
import xmltodict
import json

dict_data = xmltodict.parse(xml_data)
json_data = json.dumps(dict_data)
print(json_data)

{"feed": {"@xmlns": "http://www.w3.org/2005/Atom", "link": {"@href": "http://arxiv.org/api/query?search_query%3Dcat%3Acs.SD%26id_list%3D%26start%3D0%26max_results%3D50", "@rel": "self", "@type": "application/atom+xml"}, "title": {"@type": "html", "#text": "ArXiv Query: search_query=cat:cs.SD&id_list=&start=0&max_results=50"}, "id": "http://arxiv.org/api/Uk4hOarnIHkcx3INbQB+qNBCFEo", "updated": "2024-02-21T00:00:00-05:00", "opensearch:totalResults": {"@xmlns:opensearch": "http://a9.com/-/spec/opensearch/1.1/", "#text": "13172"}, "opensearch:startIndex": {"@xmlns:opensearch": "http://a9.com/-/spec/opensearch/1.1/", "#text": "0"}, "opensearch:itemsPerPage": {"@xmlns:opensearch": "http://a9.com/-/spec/opensearch/1.1/", "#text": "50"}, "entry": [{"id": "http://arxiv.org/abs/2402.13236v1", "updated": "2024-02-20T18:50:25Z", "published": "2024-02-20T18:50:25Z", "title": "Towards audio language modeling - an overview", "summary": "Neural audio codecs are initially introduced to compress audio 

In [59]:
# get entries from the past 4 days
import datetime
today = datetime.date.today()
entries = []
for entry in dict_data['feed']['entry']:
    date = datetime.datetime.strptime(entry['published'], '%Y-%m-%dT%H:%M:%SZ').date()
    if (today - date).days <= 4: # <= to be inclusive in case we miss something posted late on the 4th day
        entries.append(entry)
print(entries)

[{'id': 'http://arxiv.org/abs/2402.13236v1', 'updated': '2024-02-20T18:50:25Z', 'published': '2024-02-20T18:50:25Z', 'title': 'Towards audio language modeling - an overview', 'summary': 'Neural audio codecs are initially introduced to compress audio data into\ncompact codes to reduce transmission latency. Researchers recently discovered\nthe potential of codecs as suitable tokenizers for converting continuous audio\ninto discrete codes, which can be employed to develop audio language models\n(LMs). Numerous high-performance neural audio codecs and codec-based LMs have\nbeen developed. The paper aims to provide a thorough and systematic overview of\nthe neural audio codec models and codec-based LMs.', 'author': [{'name': 'Haibin Wu'}, {'name': 'Xuanjun Chen'}, {'name': 'Yi-Cheng Lin'}, {'name': 'Kai-wei Chang'}, {'name': 'Ho-Lam Chung'}, {'name': 'Alexander H. Liu'}, {'name': 'Hung-yi Lee'}], 'link': [{'@href': 'http://arxiv.org/abs/2402.13236v1', '@rel': 'alternate', '@type': 'text/htm

In [60]:
for e in entries:
    print(e['title'] + ': ' + e['published'])
    print(e['summary']) 
    print('----')

Towards audio language modeling - an overview: 2024-02-20T18:50:25Z
Neural audio codecs are initially introduced to compress audio data into
compact codes to reduce transmission latency. Researchers recently discovered
the potential of codecs as suitable tokenizers for converting continuous audio
into discrete codes, which can be employed to develop audio language models
(LMs). Numerous high-performance neural audio codecs and codec-based LMs have
been developed. The paper aims to provide a thorough and systematic overview of
the neural audio codec models and codec-based LMs.
----
HiRIS: an Airborne Sonar Sensor with a 1024 Channel Microphone Array for
  In-Air Acoustic Imaging: 2024-02-20T16:03:02Z
Airborne 3D imaging using ultrasound is a promising sensing modality for
robotic applications in harsh environments. Over the last decade, several
high-performance systems have been proposed in the literature. Most of these
sensors use a reduced aperture microphone array, leading to artifac

In [61]:
len(entries)

19

In [62]:
# use ollama / mistral to get a summary of the papers
# https://ollama.com/
# https://github.com/ollama/ollama
# https://github.com/ollama/ollama-python
# https://wandb.ai/byyoung3/ml-news/reports/How-to-Run-Mistral-7B-on-an-M1-Mac-With-Ollama--Vmlldzo2MTg4MjA0


import ollama

response = ollama.chat(model='mistral', messages=[
    {
        'role' : 'user',
        'content' : 'Please write one sentence summary of the following article description' + entries[0]['summary'] ,
    }
])

print(entries[0]['summary'])
print('----')
print(response['message']['content'])

Neural audio codecs are initially introduced to compress audio data into
compact codes to reduce transmission latency. Researchers recently discovered
the potential of codecs as suitable tokenizers for converting continuous audio
into discrete codes, which can be employed to develop audio language models
(LMs). Numerous high-performance neural audio codecs and codec-based LMs have
been developed. The paper aims to provide a thorough and systematic overview of
the neural audio codec models and codec-based LMs.
----
 This article discusses how neural audio codecs, initially designed for compressing audio data, are now being explored as effective tokenizers for creating discrete codes for developing audio language models, with several high-performance neural audio codecs and codec-based models having been developed, and the paper aims to provide an exhaustive review of these models.


In [63]:
# use ollama / mistral to get a summary of all the papers in entries
# trying with generate / prompt instead of chat
class Summary:
    def __init__(self, title, summary, generated_summary_sentence, link):
        self.title = title
        self.summary = summary
        self.generated_summary_sentence = generated_summary_sentence
        self.link = link

summaries = []

for e in entries:
    response = ollama.generate(model='mistral', 
                           prompt='Please generate a one sentence summary of the following article description: ' + e['summary'] ,
            )
    
    s = Summary(
        title = e['title'].replace('\n', ''), # remove newlines from titles
        summary = e['summary'],
        generated_summary_sentence = response['response'],
        link = e['link'][0]['@href']
    )    
    summaries.append(s)

for s in summaries:
    print(s.title)
    print(s.generated_summary_sentence)
    print(s.link)
    print('----')

Towards audio language modeling - an overview
 Researchers have discovered that neural audio codecs, initially developed for compressing audio data, can also function as effective tokenizers for creating discrete codes for audio language models, leading to the development of numerous high-performance models. (The paper aims to provide a comprehensive overview of these neural audio codec models and codec-based language models.)
http://arxiv.org/abs/2402.13236v1
----
HiRIS: an Airborne Sonar Sensor with a 1024 Channel Microphone Array for  In-Air Acoustic Imaging
 This paper introduces a high-performance in-air ultrasound sensor with 1024 microphones in a uniform rectangular array, coupled with distributed embedded hardware and advanced signal processing techniques to create artifact-free 2D and 3D ultrasound images in harsh environments.
http://arxiv.org/abs/2402.13110v1
----
Not All Weights Are Created Equal: Enhancing Energy Efficiency in  On-Device Streaming Speech Recognition
 This 

In [64]:
import requests

DISCORD_WEBHOOK_URL = '' # fill this in with your discord webhook url

for s in summaries:
    data = {
        "content" : '**' + s.title + '**' + '\n\n' + s.generated_summary_sentence + '\n\n' + s.link
    }
    requests.post(DISCORD_WEBHOOK_URL, data=data)